In [1]:
import string, re
from collections import defaultdict
import random
import copy
import math

In [2]:
def tokenize(text):
    tokens = re.findall(r"[\w]+|[^\s\w]", text)
    return tokens

def ngrams(n, tokens):
    prepend_token = '<START>'
    append_token = '<END>'
    for i in range(n - 1):
        tokens = [prepend_token] + tokens
    tokens.append(append_token)
    grams = []
    for j in range(len(tokens)-n+1):
        context = tuple(tokens[j:j+n-1])
        token = tokens[j+n-1]
        grams.append((context, token))
    return grams

class NgramModel(object):

    def __init__(self, n):
        self.n = n
        self.context_counts = defaultdict(lambda:0) # frequency of context
        self.sequence_counts = defaultdict(lambda:0) # frequency of (context, token) sequence

    def update(self, sentence):
        all_ngrams = ngrams(self.n, tokenize(sentence))
        for (context, token) in all_ngrams:
            self.context_counts[context] += 1 # increment the context count
            self.sequence_counts[(context, token)] += 1 # increment the (context, token) sequence count

    def prob(self, context, token):
        denominator = self.context_counts[context] # frequency of context followed by any token
        numerator = self.sequence_counts[(context,token)] # frequency of exact (context, token) sequence
        prob = numerator/denominator
        return prob

    def random_token(self, context):
        r = random.random()
        tokens_in_context = [] # collect all tokens that share the context
        for k, v in self.sequence_counts.items():
            # k is a (context, token) tuple
            # v is the count
            if k[0] == context: # found an ngram with same context
                tokens_in_context.append(k[1])
        tokens_in_context = sorted(tokens_in_context) # enforce natural Pythonic ordering by sorting
        pre_sum = 0
        for i, token in enumerate(tokens_in_context):
            # pre_sum = sum([self.prob(context, other_token) for other_token in tokens_in_context[:i]])
            # pre_sum is sum of probabilities up to, but excluding, the current token
            post_sum = pre_sum + self.prob(context, token)
            # post_sum also includes the probability of the current token
            if pre_sum <= r < post_sum:
                return token
            pre_sum = post_sum
    
    def random_text(self, token_count):
        output_text = []
        all_context = ['<START>' for i in range(self.n - 1)] # keep a running context list initialized with <START>s
        for i in range(token_count):
            curr_context = tuple(all_context[len(all_context)-self.n+1:]) # extract context from running context list
            next_token = self.random_token(curr_context)
            output_text.append(next_token)
            if next_token == "<END>":
                # if next token is <END> append multiple <START>s to the running context list
                for i in range(self.n - 1):
                    all_context.append('<START>')
            else:
                # otherwise append the latest token to the running context list
                all_context.append(next_token)
        return " ".join(output_text)

    def perplexity(self, sentence):
        tokens = tokenize(sentence)
        m = len(tokens)
        all_ngrams = ngrams(self.n, tokens)
        log_prob_sum = 0
        for (context, token) in all_ngrams:
            log_prob_sum += math.log(self.prob(context, token))
        perplexity = math.exp(-1/(m+1) * log_prob_sum)
        return perplexity

def create_ngram_model(n, path):
    m = NgramModel(n)
    with open(path) as f:
        for line in f:
            m.update(line)
    return m

In [14]:
path = '/Users/sppatankar/Desktop/CIS 521/Homework 8/frankenstein.txt'
n = 20
m = create_ngram_model(n, path)

In [19]:
m.random_text(500)

'" Thus I relieve thee , my creator , " he said , and placed his hated hands before my eyes , which I flung from me with violence ; " thus I take from thee a sight which you abhor . <END> She seemed pleased and went into the garden for some roots and plants , which she placed in water , and then upon the fire . <END> Who can describe their horror and consternation on beholding me ? <END> I had determined at one time that the memory of these evils should die with me , but you have won me to alter my determination . <END> During my youthful days discontent never visited my mind , and if I was ever overcome by ennui , the sight of what is beautiful in nature or the study of what is excellent and sublime in the productions of man could always interest my heart and communicate elasticity to my spirits . <END> No ! <END> Felix and Agatha spent more time in amusement and conversation , and were assisted in their labours by servants . <END> When I thought of my friends , of the mild voice of D